In [1]:
%load_ext autoreload
%autoreload 2
    
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import healpy as hp

from msfm.utils import files, catalog

# maglim lens galaxy density
For the DES Y3 data, we remove part of the survey area with our masking, removing some galaxies. Within our forward model, we only populate the masked region. We need to ensure that the galaxy counts match in both cases.

See https://arxiv.org/pdf/2105.13546 for the full maglim sample

In [2]:
conf = "/global/homes/a/athomsen/multiprobe-simulation-forward-model/configs/v16/default.yaml"
conf = files.load_config(conf)
n_side = conf["analysis"]["n_side"]

mask = files.get_tomo_masks(conf, nest_out=False)["maglim"].astype(bool)
assert np.all(mask == mask[:,[0]]),  "We assume the masks to be identical"
mask = mask[:,0]

In [3]:
des_gc_map = catalog.build_maglim_map_from_cat(conf)

26-02-16 07:21:21   catalog.py WAR   Compare with Table 1 in https://arxiv.org/pdf/2105.13546 
26-02-16 07:21:21   catalog.py INF   Maglim bin 1 
26-02-16 07:21:21   catalog.py INF   N_gal = 2236473, n_eff = 0.150 [arcmin^-2] 
26-02-16 07:21:21   catalog.py INF   Maglim bin 2 
26-02-16 07:21:21   catalog.py INF   N_gal = 1599500, n_eff = 0.107 [arcmin^-2] 
26-02-16 07:21:22   catalog.py INF   Maglim bin 3 
26-02-16 07:21:22   catalog.py INF   N_gal = 1627413, n_eff = 0.109 [arcmin^-2] 
26-02-16 07:21:22   catalog.py INF   Maglim bin 4 
26-02-16 07:21:22   catalog.py INF   N_gal = 2175184, n_eff = 0.146 [arcmin^-2] 


In [4]:
print(f"Before masking: N_gal (per-bin) = {np.sum(des_gc_map, axis=0)}")

des_gc_footprint = des_gc_map[mask]
print(f"After masking: N_gal (per-bin) = {np.sum(des_gc_footprint, axis=0)}")

Before masking: N_gal (per-bin) = [2236473. 1599500. 1627413. 2175184.]
After masking: N_gal (per-bin) = [2059105. 1470563. 1495888. 1997206.]


In [5]:
mask_area = hp.nside2pixarea(n_side, degrees=True) * np.sum(mask)
print(f"The masked footprint has area {mask_area:.2f} deg^2")

n_gal = np.sum(des_gc_footprint, axis=0) / mask_area
print(f"n_gal = {n_gal/60**2} [arcmin^-2]")
print(f"n_gal = {n_gal} [deg^-2]")

The masked footprint has area 4178.32 deg^2
n_gal = [0.137 0.098 0.099 0.133] [arcmin^-2]
n_gal = [492.807 351.951 358.012 477.993] [deg^-2]


# metacal source galaxy density
Analogous to the above for maglim

In [6]:
_, des_wl_count_map = catalog.build_metacal_map_from_cat(conf)

26-02-16 07:21:47   catalog.py WAR   Compare with Table 1 in https://arxiv.org/pdf/2105.13543 
26-02-16 07:21:47   catalog.py INF   Metacalibration bin 1 
26-02-16 07:21:47   catalog.py INF   N_gal = 24940465, n_eff = 1.476 [arcmin^-2] 
26-02-16 07:21:48   catalog.py INF   mean(e1) = 3.22e-04, mean(e2) = 1.60e-04 
26-02-16 07:21:49   catalog.py INF   sigma_e (H12) = 0.245, sigma_e (C13) = 0.187 
26-02-16 07:21:58   catalog.py INF   mean(R_gamma) = 0.7636 
26-02-16 07:22:01   catalog.py INF   z_mean (DNF) = 0.3305 
26-02-16 07:22:23   catalog.py INF   Metacalibration bin 2 
26-02-16 07:22:23   catalog.py INF   N_gal = 25280405, n_eff = 1.479 [arcmin^-2] 
26-02-16 07:22:24   catalog.py INF   mean(e1) = 3.36e-04, mean(e2) = 3.74e-05 
26-02-16 07:22:25   catalog.py INF   sigma_e (H12) = 0.265, sigma_e (C13) = 0.191 
26-02-16 07:22:30   catalog.py INF   mean(R_gamma) = 0.7182 
26-02-16 07:22:32   catalog.py INF   z_mean (DNF) = 0.5133 
26-02-16 07:22:51   catalog.py INF   Metacalibration bi

In [7]:
print(f"Before masking: N_gal (per-bin) = {np.sum(des_wl_count_map, axis=0)}")

des_wl_count_footprint = des_wl_count_map[mask]
print(f"After masking: N_gal (per-bin) = {np.sum(des_wl_count_footprint, axis=0)}")

Before masking: N_gal (per-bin) = [24940465 25280405 24891859 25091297]
After masking: N_gal (per-bin) = [23004954 23338565 22974233 23309521]


In [8]:
mask_area = hp.nside2pixarea(n_side, degrees=True) * np.sum(mask)
print(f"The masked footprint has area {mask_area:.2f} deg^2")

n_gal = np.sum(des_wl_count_footprint, axis=0) / mask_area
print(f"n_gal = {n_gal/60**2} [arcmin^-2]")
print(f"n_gal = {n_gal} [deg^-2]")

The masked footprint has area 4178.32 deg^2
n_gal = [1.529 1.552 1.527 1.55 ] [arcmin^-2]
n_gal = [5505.792 5585.636 5498.44  5578.685] [deg^-2]
